In [1]:
import pandas as pd
import numpy as np
import itertools
import inspect

# Part I - The Code For The Solution

## The Pieces Metrixes Building Blocks

#### Requiered General Data for Piece

In [2]:
def piece_data(piece_mtx):
    
    max_row = 0
    max_col = 0

    for row in range(piece_mtx.shape[0]):
        for col in range(piece_mtx.shape[1]):
            if piece_mtx.iloc[row,col]==1:
                if max_col < col:
                    max_col = col
                if max_row < row:
                    max_row = row
    num_positions = (piece_mtx.shape[0]-max_row)*(piece_mtx.shape[1]-max_col)
    max_row +=1
    max_col +=1
    size = piece_mtx.sum().sum()
    max_row_moves = piece_mtx.shape[0]-max_row
    max_col_moves = piece_mtx.shape[1]-max_col
    
    return size, max_row, max_col, num_positions, max_row_moves, max_col_moves

#### Function that moves piece i steps (rows) down and j steps (columns) right

In [3]:
def move_piece(orig_piece_mtx, num_rows_down, num_cols_right):
    max_rows_down = orig_piece_mtx.shape[0]-piece_data(orig_piece_mtx)[1]
    max_cols_right = orig_piece_mtx.shape[1]-piece_data(orig_piece_mtx)[2]
    if num_rows_down > max_rows_down:
        print ('num rows down to large, must be less then', max_rows_down+1)
        return
    if num_cols_right > max_cols_right:
        print ('num cols right to large, must be less then', max_cols_right+1)
        return

    temp_dest_piece_mtx = pd.DataFrame(np.zeros([orig_piece_mtx.shape[0], orig_piece_mtx.shape[1]], dtype = int))
    dest_piece_mtx = pd.DataFrame(np.zeros([orig_piece_mtx.shape[0], orig_piece_mtx.shape[1]], dtype = int))

    for row in range(orig_piece_mtx.shape[0]-num_rows_down):
        for col in range(orig_piece_mtx.shape[1]):
            temp_dest_piece_mtx.iat[row+num_rows_down,col] = orig_piece_mtx.iloc[row,col]


    for row in range(orig_piece_mtx.shape[0]):
        for col in range(orig_piece_mtx.shape[1]-num_cols_right):
            dest_piece_mtx.iat[row,col+num_cols_right] = temp_dest_piece_mtx.iloc[row,col]
    
    return dest_piece_mtx

#### Function that stores in a dictionary all the possible positioins of a certain piece

In [4]:
def piece_positions_mtx(orig_piece_mtx):
    piece_pos_mtx = {}
    
    for row in range((orig_piece_mtx.shape[0]-piece_data(orig_piece_mtx)[1])+1):
        for col in range((orig_piece_mtx.shape[1]-piece_data(orig_piece_mtx)[2])+1):
            piece_pos_mtx[str(row)+str(col)] = move_piece(orig_piece_mtx,row, col)
    
    return piece_pos_mtx

#### Function that calculates all the possible combinations between pair of matrixes vectors

In [5]:
def comb_list(vec_a, vec_b, vec_a_name, vec_b_name):
    list_a = []
    list_b = []
    for i in piece_positions_mtx(vec_a).keys():
        list_a.append(vec_a_name+'_'+str(i))

    for i in piece_positions_mtx(vec_b).keys():
        list_b.append(vec_b_name+'_'+str(i))

    comb_list_all = list(itertools.product(list_a, list_b))

    return (comb_list_all)

def comb_list_round(vec_a, vec_b, vec_b_name):
    list_a = []
    list_b = []
    
    list_a = vec_a.keys()
    
    for i in piece_positions_mtx(vec_b).keys():
        list_b.append(vec_b_name+'_'+str(i))   
        
    comb_list_all = list(itertools.product(list_a, list_b))
    
    return ( comb_list_all )

def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var][-1]

def comb_list_temp(vec_a, vec_b):
    list_a = []
    list_b = []
    vec_a_name = retrieve_name(vec_a)
    vec_b_name = retrieve_name(vec_b)
    
    for i in piece_positions_mtx(vec_a).keys():
        list_a.append(vec_a_name+'_'+str(i))

    for i in piece_positions_mtx(vec_b).keys():
        list_b.append(vec_b_name+'_'+str(i))

    comb_list_all = list(itertools.product(list_a, list_b))

    return (comb_list_all)

#### Function that returns dictionary of the valid combinations (i.e. pices do not cover one another) from all the possible combinations between pair of matrixes vectors, the weight (i.e. the num of covered squares) and the combined valid matrixes.

In [6]:
def valid_comb (vec_a, vec_b, vec_a_name, vec_b_name):
    
    comb_list_all = comb_list(vec_a, vec_b, vec_a_name, vec_b_name)
    valid_comb_list = comb_list_all[:]
    puzzle_dict = {}
    
    for comb in comb_list_all:
        #print (comb)
        val1_0 = comb[0].split('_')[0]
        val1_1 = comb[0].split('_')[1]

        val2_0 = comb[1].split('_')[0]
        val2_1 = comb[1].split('_')[1]
    
        if (val1_0 == vec_a_name) and (val2_0 == vec_b_name):
            
            comb_puzzle = piece_positions_mtx(vec_a)[val1_1] + piece_positions_mtx(vec_b)[val2_1]
            
            weight = 0
            for row in comb_puzzle.index:
                for col in comb_puzzle.columns:            
                    weight = weight + comb_puzzle.iloc[row,col]
            
            puzzle_dict[comb] = weight, comb_puzzle
            #print ( puzzle_dict )
        
            for row in comb_puzzle.index:
                for col in comb_puzzle.columns:
                    if comb_puzzle.iloc[row,col] > 1:
                        puzzle_dict.pop(comb)
                        valid_comb_list.remove(comb)
                        break
                if comb_puzzle.iloc[row,col] > 1:
                    break
    
    return puzzle_dict, valid_comb_list

def valid_comb_round (vec_a, vec_b, vec_b_name):
    comb_list_all = comb_list_round(vec_a, vec_b, vec_b_name)
    valid_comb_list = comb_list_all[:]
    puzzle_dict = {}
    
    for comb in comb_list_all:
        #print (comb)
        val2_0 = comb[1].split('_')[0]
        val2_1 = comb[1].split('_')[1]
    
        comb_puzzle = vec_a[comb[0]][1] + piece_positions_mtx(vec_b)[val2_1]
        comb_puzzle_before = vec_a[comb[0]][1]
        weight_puzzle_before = vec_a[comb[0]][0]
        #print (comb_puzzle)
        
        weight = 0
        for row in comb_puzzle.index:
            for col in comb_puzzle.columns:            
                weight = weight + comb_puzzle.iloc[row,col]
            
        puzzle_dict[comb] = weight, comb_puzzle
        #print ( puzzle_dict )
        
        for row in comb_puzzle.index:
            for col in comb_puzzle.columns:
                if comb_puzzle.iloc[row,col] > 1:
                    puzzle_dict.pop(comb)
                    puzzle_dict[comb[0]] = weight_puzzle_before, comb_puzzle_before
                    valid_comb_list.remove(comb)
                    if comb[0] not in valid_comb_list:
                        valid_comb_list.append(comb[0])
                    break
            if comb_puzzle.iloc[row,col] > 1:
                break
    
    return puzzle_dict, valid_comb_list

# Part II - Running the Algo.

### 1) Load and Prepare The Data

In [7]:
file_name_string = 'C:/Users/Hanan/Desktop/Data Science bootcamp/Puzzle/PuzzlePiecesLight6.xlsx'

In [8]:
df_sheet_map = pd.read_excel(file_name_string, sheet_name=None, index_col=None, na_values=['NA'], header=None)

In [9]:
piece01 = df_sheet_map['Sheet1']
piece02 = df_sheet_map['Sheet2']
piece03 = df_sheet_map['Sheet3']
piece04 = df_sheet_map['Sheet4']
piece05 = df_sheet_map['Sheet5']
piece06 = df_sheet_map['Sheet6']
piece07 = df_sheet_map['Sheet7']
piece08 = df_sheet_map['Sheet8']
piece09 = df_sheet_map['Sheet9']
piece10 = df_sheet_map['Sheet10']
piece11 = df_sheet_map['Sheet11']
piece12 = df_sheet_map['Sheet12']
piece13 = df_sheet_map['Sheet13']
piece14 = df_sheet_map['Sheet14']
piece15 = df_sheet_map['Sheet15']
piece16 = df_sheet_map['Sheet16']

In [10]:
pieces_lst = ['piece01', 'piece02', 'piece03', 'piece04', 'piece05', 'piece06', 'piece07', 'piece08', 'piece09', \
              'piece10', 'piece11', 'piece12', 'piece13', 'piece14', 'piece15', 'piece16']
pieces = [piece01, piece02, piece03, piece04, piece05, piece06, piece07, piece08, piece09, piece10, \
         piece11, piece12, piece13, piece14, piece15, piece16]

In [11]:
sum = 0
for piece in pieces:
    sum = sum + piece.sum().sum()
print ( sum )

100


#### create a dictionary of the pieces sorted by their number of posibble possitions on the board. In order to reduce complexity it's recomended to start with the less positions ones.

In [12]:
size_dict = {}
for piece in pieces_lst:
    size_dict[piece] = piece_data(vars()[piece])[3], piece_data(vars()[piece])[0]

sorted_piece_size=sorted(size_dict.items(),key=lambda x:x[1][0],reverse=False)
sorted_piece_size

[('piece09', (6, 8)),
 ('piece07', (8, 9)),
 ('piece11', (8, 8)),
 ('piece02', (9, 9)),
 ('piece16', (9, 7)),
 ('piece05', (10, 7)),
 ('piece01', (12, 7)),
 ('piece03', (12, 7)),
 ('piece14', (12, 8)),
 ('piece10', (15, 6)),
 ('piece08', (16, 6)),
 ('piece06', (16, 5)),
 ('piece04', (20, 4)),
 ('piece15', (24, 3)),
 ('piece12', (25, 3)),
 ('piece13', (25, 3))]

### 2) Run the Algo. Rounds Rercusivly

#### Round-1: Btween piece09 & piece07 (The pieces with the less possible board possitions)

In [18]:
round1 = valid_comb(piece09, piece07, 'piece09', 'piece07')[0]
sorted(round1.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(('piece09_10', 'piece07_00'), (17,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  0  0  0
   3  0  0  1  0  0  0
   4  0  0  1  1  0  0
   5  1  1  1  0  0  0))]

#### Round-2: Btween Round-1 & piece11 (pieces: 09, 07, 11)

In [19]:
round2 = valid_comb_round(round1, piece11, 'piece11')[0]
sorted(round2.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(('piece09_10', 'piece07_00'), (17,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  0  0  0
   3  0  0  1  0  0  0
   4  0  0  1  1  0  0
   5  1  1  1  0  0  0))]

#### Round-3: Btween Round-2 & piece02 (pieces: 09, 07, 11, 02)

In [20]:
round3 = valid_comb_round(round2, piece02, 'piece02')[0]
sorted(round3.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(('piece09_10', 'piece07_00'), (17,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  0  0  0
   3  0  0  1  0  0  0
   4  0  0  1  1  0  0
   5  1  1  1  0  0  0))]

#### Round-4: Btween Round-3 & piece16 (pieces: 09, 07, 11, 02, 16)

In [21]:
round4 = valid_comb_round(round3, piece16, 'piece16')[0]
sorted(round4.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(('piece09_10', 'piece07_00'), (17,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  0  0  0
   3  0  0  1  0  0  0
   4  0  0  1  1  0  0
   5  1  1  1  0  0  0))]

#### Round-5: Btween Round-4 & piece05 (pieces: 09, 07, 11, 02, 16, 05)

In [22]:
round5 = valid_comb_round(round4, piece05, 'piece05')[0]
sorted(round5.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((('piece09_10', 'piece07_00'), 'piece05_04'), (24,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  0  0  1
   3  0  0  1  0  0  1
   4  0  0  1  1  1  1
   5  1  1  1  0  0  0))]

#### Round-6: Btween Round-5 & piece01 (pieces: 09, 07, 11, 02, 16, 05, 01)

In [23]:
round6 = valid_comb_round(round5, piece01, 'piece01')[0]
sorted(round6.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((('piece09_10', 'piece07_00'), 'piece05_04'), (24,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  0  0  1
   3  0  0  1  0  0  1
   4  0  0  1  1  1  1
   5  1  1  1  0  0  0))]

#### Round-7: Btween Round-6 & piece03 (pieces: 09, 07, 11, 02, 16, 05, 01, 03)

In [24]:
round7 = valid_comb_round(round6, piece03, 'piece03')[0]
sorted(round7.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((('piece09_10', 'piece07_00'), 'piece05_04'), (24,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  0  0  1
   3  0  0  1  0  0  1
   4  0  0  1  1  1  1
   5  1  1  1  0  0  0))]

#### Round-8: Btween Round-7 & piece14 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14)

In [25]:
round8 = valid_comb_round(round7, piece14, 'piece14')[0]
sorted(round8.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((('piece09_10', 'piece07_00'), 'piece14_23'), (25,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  1  1  0
   3  0  0  1  0  1  1
   4  0  0  1  1  0  1
   5  1  1  1  1  1  1))]

#### Round-9: Btween Round-8 & piece10 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10)

In [26]:
round9 = valid_comb_round(round8, piece10, 'piece10')[0]
sorted(round9.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((('piece09_10', 'piece07_00'), 'piece14_23'), (25,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  1  1  0
   3  0  0  1  0  1  1
   4  0  0  1  1  0  1
   5  1  1  1  1  1  1))]

#### Round-10: Btween Round-9 & piece08 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10, 08)

In [27]:
round10 = valid_comb_round(round9, piece08, 'piece08')[0]
sorted(round10.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((('piece09_10', 'piece07_00'), 'piece14_23'), (25,    0  1  2  3  4  5
   0  0  1  1  1  0  0
   1  1  1  1  1  1  0
   2  1  1  1  1  1  0
   3  0  0  1  0  1  1
   4  0  0  1  1  0  1
   5  1  1  1  1  1  1))]

#### Round-11: Btween Round-10 & piece06 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10, 08, 06)

In [28]:
round11 = valid_comb_round(round10, piece06, 'piece06')[0]
sorted(round11.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(((('piece09_11', 'piece07_01'), 'piece10_24'), 'piece06_30'),
  (28,    0  1  2  3  4  5
   0  0  0  1  1  1  0
   1  0  1  1  1  1  1
   2  0  1  1  1  1  0
   3  0  0  1  1  1  1
   4  1  1  1  1  1  1
   5  1  1  1  1  1  1))]

#### Round-12: Btween Round-11 & piece04 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10, 08, 06, 04)

In [29]:
round12 = valid_comb_round(round11, piece04, 'piece04')[0]
sorted(round12.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(((('piece09_10', 'piece07_00'), 'piece14_23'), 'piece04_04'),
  (29,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  1  1
   3  0  0  1  0  1  1
   4  0  0  1  1  0  1
   5  1  1  1  1  1  1))]

#### Round-13: Btween Round-12 & piece15 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10, 08, 06, 04, 15)

In [30]:
round13 = valid_comb_round(round12, piece15, 'piece15')[0]
sorted(round13.items(),key=lambda x:x[1][0],reverse=True)[:1]

[(((('piece09_10', 'piece07_00'), 'piece14_23'), 'piece04_04'),
  (29,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  1  1
   3  0  0  1  0  1  1
   4  0  0  1  1  0  1
   5  1  1  1  1  1  1))]

#### Round-14: Btween Round-13 & piece12 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10, 08, 06, 04, 15, 12)

In [31]:
round14 = valid_comb_round(round13, piece12, 'piece12')[0]
sorted(round14.items(),key=lambda x:x[1][0],reverse=True)[:1]

[((((('piece09_10', 'piece07_00'), 'piece14_23'), 'piece04_04'), 'piece12_30'),
  (32,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  1  1
   3  1  0  1  0  1  1
   4  1  1  1  1  0  1
   5  1  1  1  1  1  1))]

#### Round-15: Btween Round-14 & piece13 (pieces: 09, 07, 11, 02, 16, 05, 01, 03, 14, 10, 08, 06, 04, 15, 12, 13)

In [32]:
round15 = valid_comb_round(round14, piece13, 'piece13')[0]
sorted(round15.items(),key=lambda x:x[1][0],reverse=True)[:5]

[(((((('piece09_10', 'piece07_00'), 'piece10_23'), 'piece04_24'),
    'piece12_30'),
   'piece13_04'),
  (33,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  1  1
   3  1  0  1  1  1  1
   4  1  1  1  1  1  1
   5  1  1  1  1  1  0)),
 (((((('piece09_10', 'piece07_00'), 'piece05_04'), 'piece15_53'),
    'piece12_30'),
   'piece13_23'),
  (33,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  1  1
   3  1  0  1  0  1  1
   4  1  1  1  1  1  1
   5  1  1  1  1  1  1)),
 (((((('piece09_10', 'piece07_00'), 'piece05_04'), 'piece15_53'),
    'piece12_23'),
   'piece13_30'),
  (33,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  0  1
   3  1  1  1  1  1  1
   4  0  1  1  1  1  1
   5  1  1  1  1  1  1)),
 ((((('piece09_10', 'piece07_00'), 'piece14_23'), 'piece04_04'), 'piece12_30'),
  (32,    0  1  2  3  4  5
   0  0  1  1  1  1  1
   1  1  1  1  1  1  1
   2  1  1  1  1  1  1
   3  1  0 

# Part III - The Solution combinations

Each sequence represents a possible optimal solution.
The solution's layot on the board can be understood from the pieces name syntax :
PieceNum_Num rows down from orig. pos., Num columns right from orig. pos.

'piece09_10' --> piece number 9. one row down from orig, aligned to the left.
'piece07_00' --> piece number 7, at origin position.

In [35]:
temp = sorted(round15.items(),key=lambda x:x[1][0],reverse=True)[:3]
solutions = [ [comb[0], comb[1][0]] for comb in temp ]
solutions

[[((((('piece09_10', 'piece07_00'), 'piece10_23'), 'piece04_24'),
    'piece12_30'),
   'piece13_04'),
  33],
 [((((('piece09_10', 'piece07_00'), 'piece05_04'), 'piece15_53'),
    'piece12_30'),
   'piece13_23'),
  33],
 [((((('piece09_10', 'piece07_00'), 'piece05_04'), 'piece15_53'),
    'piece12_23'),
   'piece13_30'),
  33]]